# # Download & Install the packages

In [ ]:
%%capture
!git clone https://github.com/jacons/ilmart
!git clone --recurse-submodules https://github.com/veneres/LightGBM.git
!pip install rankeval
%cd LightGBM/python-package
!python setup.py install
%cd ../../ilmart
!pip install -e .
%cd ../ilmart

In [ ]:
import sys
import pandas as pd
import numpy as np

from tqdm import tqdm
from typing import Tuple
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import ndcg_score
from src.ilmart import Ilmart, IlmartDistill, utils
import lightgbm as lgbm

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
%reload_ext autoreload
%autoreload 2

# # Grid search implementation

In [ ]:
class ILMARTGridsearch:
  def __init__(self, name: str, path: str = None, nDCG_at: int = 15):

    self.train = pd.read_csv(f"{path}{name}_dataset_tr.csv")
    self.valid = pd.read_csv(f"{path}{name}_dataset_vl.csv")
    self.test = pd.read_csv(f"{path}{name}_dataset_ts.csv")

    # sorting after the splitting
    self.train.sort_values(["qId", "kId"], inplace=True)
    self.valid.sort_values(["qId", "kId"], inplace=True)
    self.test.sort_values(["qId", "kId"], inplace=True)

    # Preparing the datasets
    self.qIds_train = self.train.groupby("qId")["qId"].count().to_numpy()
    self.X_train, self.y_train = self.train.iloc[:, 5:], self.train[["qId", "kId", "binned_relevance"]]
    self.qIds_valid = self.valid.groupby("qId")["qId"].count().to_numpy()
    self.X_valid, self.y_valid = self.valid.iloc[:, 5:], self.valid[["qId", "kId", "binned_relevance"]]
    self.qIds_test = self.test.groupby("qId")["qId"].count().to_numpy()
    self.X_test, self.y_test = self.test.iloc[:, 5:], self.test[["qId", "kId", "binned_relevance"]]

    self.default_par = dict(
      objective="lambdarank",
      boosting_type="gbdt",
      metric="ndcg",
      force_row_wise=True,
      n_jobs=-1,
      verbose = -1
    )
    self.ranker_par = dict(  # default ranker parameters (used in fitting) pt.2
      X_train=self.X_train.to_numpy(),
      y_train=self.y_train["binned_relevance"].to_numpy(),
      group_train=self.qIds_train,
      X_valid = self.X_valid.to_numpy(),
      y_valid = self.y_valid["binned_relevance"].to_numpy(),
      group_valid=self.qIds_valid,
    )

    self.nDCG_at = nDCG_at
    return

  def eval_model(self, model, df: pd.DataFrame = None,
                 qIds: np.ndarray = None, nDCG_at: list = None) -> dict:
      """
      Custom evaluation function: the function groups by the "job-offers" and foreach set, it predicts
      the "lambdas" that it uses to sort (by binned_relevance).
      After obtained nDCGs apply the average.
      """
      df = self.valid if df is None else df
      n_qIds = len(self.qIds_valid) if qIds is None else len(qIds)
      nDCG_at = [self.nDCG_at] if nDCG_at is None else nDCG_at
      avg_nDCG = np.zeros((len(nDCG_at)))

      for _, v in df.groupby("qId"):

          features, target = v.iloc[:, 5:].to_numpy(), np.asarray([v["relevance"].to_numpy()])
          lambdas = np.asarray([model.get_model().predict(features)])  # predict lambdas

          # Perform the nDCG for a specific job-offer and then sum it into cumulative nDCG
          for i, nDCG in enumerate(nDCG_at):
              avg_nDCG[i] += ndcg_score(target, lambdas, k=nDCG)

      # dived by the number of jobs-offer to obtain the average.
      avg_nDCG /= n_qIds
      results = {"nDCG@"+str(nDCG): round(avg_nDCG[i], 4) for i, nDCG in enumerate(nDCG_at)}
      return results

  def fit(self, **conf):

      model = Ilmart(verbose=False)
      all_params = { **self.default_par, **conf}
      model.fit(all_params,**self.ranker_par)
      return model

  def grid_search(self, hyperparameters: dict = None) -> Tuple:
      # keep the current: (best_model, best_params, best nDCG)
      best_model_: Tuple = (None, None, -sys.maxsize)

      # explore all possible combinations of hyperparameters
      progress_bar = tqdm(ParameterGrid(hyperparameters))
      for conf in progress_bar:
          model = self.fit(**conf)
          avg_nDCG = self.eval_model(model)["nDCG@"+str(self.nDCG_at)]

          # if the model is better respect to the previous one, it updates the tuple
          if avg_nDCG > best_model_[2]:
              best_model_ = (model, conf, avg_nDCG)
          progress_bar.set_postfix(nDCG_15=best_model_[2])
      return best_model_

In [ ]:
ilMart_parameter = dict(
    num_leaves=[2, 3 , 4, 5, 6],
    max_depth=[2, 3],
    learning_rate=[0.02, 0.05, 0.08, 0.1, 0.15, 0.2],
    reg_lambda=[0.00005, 0.0001, 0.0002, 0.0003]
)
gridsearch_parameters = dict(
    path="https://raw.githubusercontent.com/jacons/Interpretable-by-design-Human-Recommender/master/outputs/scores/",
    name="0",
    nDCG_at=15
)

In [ ]:
gs = ILMARTGridsearch(**gridsearch_parameters)
best_ = gs.grid_search(ilMart_parameter)

In [ ]:
best_

In [ ]:
# nDCG
nDCG_train = gs.eval_model(model=best_[0], df=gs.train, qIds=gs.qIds_train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_[0], df=gs.valid, qIds=gs.qIds_valid, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_[0], df=gs.test, qIds=gs.qIds_test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

In [ ]:
best_[0].get_model().save_model("file_name.lgbm")
best_model = lgbm.Booster(model_file="file_name.lgbm")
distilled_best_model = IlmartDistill(best_model)

# # Feature importance

In [ ]:
uri2feature = {i:v for i,v in enumerate(list(gs.X_train.columns))}

In [ ]:
list_features = list(gs.X_train.columns)
features_importance = best_model.feature_importance()
order_idx = np.argsort(features_importance)
features_importance = features_importance[order_idx]
labels_name = [list_features[i] for i in order_idx]

In [ ]:
plt.barh(range(len(features_importance)), features_importance, color="#a74e25")
plt.grid(color='grey', linestyle='-.', linewidth=0.5, alpha=0.5)
plt.yticks(range(len(features_importance)), labels_name)
plt.title('Features importance')
plt.xlabel('Features importance')
plt.ylabel('Features')
plt.show()

# # Explain the model

In [ ]:
def pairwise_function(cuts, contribution, value: float) -> float:
  if value < cuts[0]:
      return contribution[0]
  for i in range(len(cuts)-1):
      if cuts[i] <= value <= cuts[i+1]:
          return contribution[i+1]
  if value > cuts[-1]:
      return contribution[-1]

def explanation(model, index_feature: int, eps: float = 0.01) -> Tuple[np.ndarray, np.ndarray]:

  min_, max_ = gs.X_train.iloc[:,index_feature].min(), gs.X_train.iloc[:,index_feature].max()
  cuts = model.splitting_values[index_feature][1:-1]
  contrib = model.hist[(index_feature,)]
  x = np.arange(min_, max_, eps)
  y = [pairwise_function(cuts, contrib, v_) for v_ in x]
  return x, y

In [ ]:
f, axs = plt.subplots(ncols=3,nrows=2,figsize=(20,8))

for i, feature in enumerate(order_idx[::-1][:6]):

    x,y = explanation(distilled_best_model, feature)
    axs[int(i/3)][i%3].set_xlabel("Value")
    axs[int(i/3)][i%3].set_ylabel("Contribute")
    axs[int(i/3)][i%3].grid()
    axs[int(i/3)][i%3].plot(x,y, label=uri2feature[feature], color="#a74e25")
    axs[int(i/3)][i%3].legend()
plt.show()

In [ ]:
gs.test[gs.test["qId"]==15][["kId","binned_relevance"]].sort_values("binned_relevance",ascending=False).head(15)

In [ ]:
features = gs.test[gs.test["qId"]==15]
y_pred = best_model.predict(np.asarray(features.iloc[:,5:].values))
y_pred = pd.DataFrame(y_pred, index=features.index, columns=["lambdas"])
dt_final = pd.merge(features, y_pred, left_index=True, right_index=True)
dt_final.sort_values("lambdas",ascending=False)[["kId","relevance"]].head(15)